# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 [Online.](https://arxiv.org/abs/1611.06455 "Wang et al. (2017)")

## Formatting

Left align the tables in this Notebook.


In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Data
Additional datasets are available in the UEA & UCT Time Series Classification Repository at http://www.timeseriesclassification.com/. 

Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

### GunPoint dataset
The GunPoint dataset is from the [UEA & UCR Time Series 
Classification Repository](http://www.timeseriesclassification.com/description.php?Dataset=GunPoint 
"GunPoint description"). The data is from one female and one male either drawing and pointing a gun at a target or pointing their finger at a target. The location of their hand was tracked. This data is the time series of the x-axis location.

|Training set size |Test set size |Number of classes 
|:-----------      |:--------     |:----------     
|50 |150  |2 


## Train MLP
Expected running time for various datasets when using the original training/test split and the original batch_size. Running on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset   | Training time
:-------  | :--------
Adiac     | 18 minutes
GunPoint  | 11 minutes


### User inputs

In [ ]:
from datetime import datetime

flist = ['private_dog0']  # List dataset directory names. WormsTwoClass Lightning2 Earthquakes GunPoint 
batch_size = 32 # Set to -1 to use Wang et al settings
nb_epochs = 1500
k = 10 # For k-fold cross validation. If k=1, the original test-train split is used.
m = 1 # Number of repetitions of k-fold cross validation (if k>1).
seed = 35 # seed for initialising random weights in the NN.
k_fold_seed = 87
tensorboard = True # Set to True to write logs for use by TensorBoard

# Directories
logs_dir = '../logs'
tensorboard_dir = '../logs/tensorboard'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp

# Input directory
if 'private' in flist[0]:
    fdir = '../data/private_data/private_events_dev' 
else:
    fdir = '../data' 

### Set up

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
#from keras import backend as K

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.metrics import confusion_matrix

import os
import pathlib
import time

np.random.seed(22)#813306)
      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  

### Visualise the data

In [ ]:
fname = flist[0]
data_a = 0
data_b = 2
x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
plt.plot(x_train[data_a], label='category'+str(y_train[data_a]))
plt.plot(x_train[data_b], label='category'+str(y_train[data_b]))
plt.legend(loc='upper left', frameon=False)
x_train.shape

### Function to train the MLP

In [ ]:
def train_model(fname, x_train, y_train, x_test, y_test, label="0"):
    print('Running dataset', fname)
    if batch_size == -1:
        batch = int(min(x_train.shape[0]/10, 16)) # Wang et al. setting.
    else:
        batch=batch_size 
    
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    # Build model
    x = Input(x_train.shape[1:])
    y= Dropout(0.1,name='Drop010')(x)
    y = Dense(500, activation='relu', name='Dense010')(x)
    y = Dropout(0.2,name='Drop020')(y)
    y = Dense(500, activation='relu', name='Dense020')(y)
    y = Dropout(0.2,name='Drop030')(y)
    y = Dense(500, activation = 'relu', name='Dense030')(y)
    y = Dropout(0.3,name='Drop040')(y)
    out = Dense(nb_classes, activation='softmax', name='Out')(y)
    model = Model(x, out)
    #print(model.summary())
    
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8)  
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    callbacks = [reduce_lr]
    if tensorboard:
        callbacks = [reduce_lr, keras.callbacks.TensorBoard(log_dir=tensorboard_dir+'/'+fname+'_'+label, histogram_freq=1)]

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
    end = time.time()
    
    log = pd.DataFrame(hist.history)   
    # Print results. Print the testing results which has the lowest training loss.
    print('Training complete on', fname)
    duration_minutes = str(round((end-start)/60))
    print('Training time ', end-start, 'seconds, which is about', duration_minutes, 'minutes.')    
    print('Selected the test result with the lowest training loss. Loss and validation accuracy are -')
    idx = log['loss'].idxmin()
    loss = log.loc[log['loss'].idxmin]['loss']
    val_acc = log.loc[log['loss'].idxmin]['val_acc']
    print(loss, val_acc, 'at index', str(idx), ' (epoch ', str(idx+1), ')')
    summary = '|' + label + '  |'+str(loss)+'  |'+str(val_acc)+' |'+str(idx)+' |'+ duration_minutes + 'mins  |'
    summary_csv = label+','+str(loss)+','+str(val_acc)+','+str(idx)+','+ duration_minutes 
    # Save summary file and log file.
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    with open(logs_dir+'/'+fname+'/summary.csv', 'a+') as f:
        f.write(summary_csv)
        f.write('\n')
        print('Added summary row to ', logs_dir+'/'+fname+'/summary.csv')  
    print('Saving logs to',logs_dir+'/'+fname+'/history_'+label+'.csv')
    log.to_csv(logs_dir+'/'+fname+'/history_'+label+'.csv')
    return summary, model
 



# Train

In [ ]:
results = []
for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    # k-fold cross validation setup
    if k > 1:
        x_all = np.concatenate((x_train, x_test), axis=0)
        y_all = np.concatenate((y_train, y_test), axis=0)
        kfold = RepeatedStratifiedKFold(n_splits=k, n_repeats=m, random_state=k_fold_seed)
        count = 0
        for train, test in kfold.split(x_all):
            x_train, y_train, x_test, y_test = x_all[train], y_all[train], x_all[test], y_all[test]
            summary, model = train_model(fname, x_train, y_train, x_test, y_test, str(count))
            results.append(summary)
            count = count + 1
    else:
        summary, model = train_model(fname, x_train, y_train, x_test, y_test)
        results.append(summary)


print('DONE')
print(fname, timestamp)
print('train:test', y_train.shape[0], y_test.shape[0])
for each in results:
    print(each)


#with tf.Graph().as_default():
#    model = main()

In [ ]:
# Completed at
'{:%Y-%m-%dT%H:%M}'.format(datetime.now())

# Recorded results
## Adiac

|Run |Loss |Accuracy | Comment
|:---|:--- |:---     |:----------
|1   | 0.005222544357037315  | 0.6930946294608933 |Adadelta - default values
|2   | 0.0018498263113997382 | 0.7109974427601261 |
|3   | 0.001620212956988372  | 0.7212276216358176 |Adadelta parameters match Wang (2017)
|4   | 0.0023210097823697976 | 0.7289002559069172  |
|5   | 0.001681140968559283   | 0.7212276216358176  |

## GunPoint

|Run |Loss |Accuracy | Comment     | Timestamp
|:---|:--- |:---     |:----------  |:--------
|1   | 1.466274483163943e-07  | 0.9400000035762787 |Adadelta parameters match Wang (2017)
|2   | 1.370907000364241e-07  | 0.9333333363135655  |
|3   | 1.3589860969887013e-07 | 0.9333333373069763  |
|4   | 1.3351442333942033e-07   | 0.9333333363135655  |
|5   | 1.4781954007503373e-07   | 0.9333333363135655  |
|6   | 1.4781953865394827e-07   | 0.940000002582868  |
|7   | 1.4305116877721957e-07   | 0.940000002582868  |
|8   | 1.239776764805356e-07   | 0.946666669845581  |
|9   | 1.311302369799705e-07   | 0.9333333363135655  | Changed random seed from 813306 to 22
|10  | 1.466274483163943e-07   | 0.9333333363135655  | Added kernel_initializer=RandomUniform(seed=seed) with seed=35
|11   | 7.161587154865266   | 0.49333334614833196  | Changed from Adadelta optimiser to Adam(lr=0.1, epsilon=1e-8)
|12   | 1.2874604919943523e-07   | 0.9266666700442632  |Adam(default params)| 2018-11-18T12:41   
|13   | 1.5139580966661015e-07   | 0.9333333363135655  | . | 2018-11-18T15:28  
|14   | 1.3470651367697427e-07   | 0.9333333363135655  |  .| 2018-11-23T12:37   |
|15  |2.695351213333197e-06  |0.9399999976158142 |Batch size 128 - 1mins  | 2018-12-02T13:07 |
|16  |2.858659172488842e-06  |0.9466666642824809 |Batch size 128 - 1mins  | 2018-12-02T13:11 |
|17  |2.6953459837386617e-06  |0.9466666642824809 |Sequential instead of Model API. Batch size 64 - 2mins  | 2018-12-02T19:21|

### K-fold cross validation
|Run |Loss |Accuracy | Comment     | Timestamp
|:---|:--- |:---     |:----------  |:--------
|1.0  | 2.058348101741103e-07   | 1.0  |  .| 2018-11-23T12:51   |
|1.1   | 2.7934738824342274e-07   | 0.9400000035762787  |  .| 2018-11-23T12:51   |
|1.2   | 2.1417950222257786e-07   | 0.9200000047683716  |  .| 2018-11-23T12:51   |
|1.3   | 1.9788750762472774e-07   | 0.9800000011920929  |  .| 2018-11-23T12:51   |

|Run |Loss |Accuracy | Dataset     | Timestamp | Duration
|:---|:--- |:---     |:----------  |:--------  |:-------------
|0  |2.5153182861004095e-07  |1.0 |GunPoint  |2018-11-23T13:45  |7mins  |
|1  |2.415976368297379e-07  |0.9600000023841858 |GunPoint  |2018-11-23T13:45  |7mins  |
|2  |1.8239028776179111e-07  |0.9400000035762787 |GunPoint  |2018-11-23T13:45  |7mins  |
|3  |2.467634772074234e-07  |0.9800000011920929 |GunPoint  |2018-11-23T13:45  |7mins  |

|Run |Loss |Accuracy | Dataset     | Timestamp | Duration
|:---|:--- |:---     |:----------  |:--------  |:-------------
|0  |2.079541167808606e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|1  |2.076229498647485e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|2  |2.1060326010532057e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|3  |1.6821761650792017e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|4  |2.0232486279938812e-07  |0.95 |GunPoint  |2018-11-23T14:38  |8mins  |
|5  |2.457037595604561e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|6  |1.8775471220225073e-07  |0.9 |GunPoint  |2018-11-23T14:38  |8mins  |
|7  |2.1126546982941364e-07  |0.9 |GunPoint  |2018-11-23T14:38  |8mins  |
|8  |1.8775469325444444e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |
|9  |2.4570393135389974e-07  |1.0 |GunPoint  |2018-11-23T14:38  |8mins  |



#### GunPoint 2018-12-02T13:15
Batch size = 64

|Run |Loss |Accuracy | Epoch index     | Duration
|:---|:--- |:---     |:----------      |:-------------
|0  |1.5788874268057246e-06  |1.0 |4637 |2mins  |
|1  |1.2871531074173011e-06  |1.0 |3602 |2mins  |
|2  |1.1083247272836161e-06  |1.0 |4959 |2mins  |
|3  |1.523910724346125e-06  |0.949999988079071 |4749 |2mins  |
|4  |2.1378607546769975e-06  |0.949999988079071 |3957 |2mins  |
|5  |1.4994056679521842e-06  |1.0 |3945 |2mins  |
|6  |1.6245767508533188e-06  |0.949999988079071 |4641 |2mins  |
|7  |1.655717910075004e-06  |0.8999999761581421 |4676 |2mins  |
|8  |1.1629627541272687e-06  |1.0 |4574 |2mins  |
|9  |1.6093625062138825e-06  |1.0 |3838 |2mins  |

# Read results from file

In [ ]:
read_log = pd.read_csv(logs_dir+'/'+fname+'/history_0.csv')
read_loss = read_log.loc[read_log['loss'].idxmin]['loss']
read_val_acc = read_log.loc[read_log['loss'].idxmin]['val_acc']
print('Read from file', logs_dir+'/'+fname+'/history_0.csv', '- loss and val_acc are')
print(read_loss, read_val_acc)

In [ ]:
file = '../logs/2019-03-16T09:23/private_dog0_correct/summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
accuracy = data.iloc[:,2]
print(data.describe())
print('Accuracy mean and 95% confidence level is', accuracy.mean(), accuracy.std()*1.96)
print('95% confidence interval is', accuracy.quantile(0.0025), 'to', accuracy.quantile(0.975))
plt.figure(0)
data.boxplot(column='loss')
plt.figure(1)
data.boxplot(column='val_acc')
data.hist(column='val_acc')
print('Rows with val_acc<.94 are')
data[data.val_acc<0.94]

In [ ]:
def plot_confusion_matrix(cm, title='Normalised confusion matrix'):
    # Plot the normalised confusion matrix
    # Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
    # 'Confusion Matrix' https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    classes = ['Positive', 'Negative']
    cmap=plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.tight_layout()

In [ ]:
# Use trained model (after all epochs) to make predictions
x_input = x_test
y_input = y_test
y_input = y_input - y_input.min()
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_input = (x_input - x_train_mean)/(x_train_std)
nb_classes = len(np.unique(y_input))
y_input = (y_input - y_input.min())/(y_input.max()-y_input.min())*(nb_classes-1)
# Calculate model prediction
y_probs = model.predict_on_batch(x_input)
y_class = y_probs.argmax(axis=1)
cm = confusion_matrix(y_input, y_probs.argmax(axis=1), labels=[1,0])
acc_calc = (cm[0][0]+cm[1][1])/(cm.sum())
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Predicted class probabilities:\n', y_probs[:5,:])
print('Pred', y_class[:20])
print('True', y_input[:20].astype(int))
print(cm)
print('Calculated accuracy:',acc_calc) 
print('Normalised confusion matrix:\n', cm_norm)
title = 'Normalised confusion matrix'
plot_confusion_matrix(cm_norm, title=title)

In [ ]:
y_test

In [ ]:
#K.clear_session()
#tf.reset_default_graph()